# 七. 误差反向传播算法

In [1]:
%config InlineBackend.figure_formats = ['svg']
%matplotlib inline
from torch.utils.data import TensorDataset, DataLoader
import torch
import torchvision.transforms as transforms
import gzip
import numpy as np
import os

上一节我们看到了神经⽹络如何使⽤梯度下降算法来学习他们⾃⾝的权重和偏置。但是，这⾥还留下了⼀个问题：我们并没有讨论如何计算代价函数的梯度。这是很⼤的缺失！我们接下来学习计算这些梯度的快速算法，也就是反向传播（`backpropagation`）。

反向传播算法最初在1970 年代被提及，但是⼈们直到David Rumelhart、Geoffrey Hinton 和Ronald Williams 的著名的1986 年的论⽂中才认识到这个算法的重要性。这篇论⽂描述了对⼀些神经⽹络反向传播要⽐传统的⽅法更快，这使得使⽤神经⽹络来解决之前⽆法完成的问题变得可⾏。现在，反向传播算法已经是神经⽹络学习的重要组成部分了。

## 0. 符号约定

<div align=center>
<img width="700" src="../pictures/7.back_propagation.svg"/>
</div>

<div align=center> 
    图1 神经网络数学符号示意图
</div>

假设输入层表示为$x$, 输入层到第1个隐藏层的权重矩阵表示为$w^1$, 第1个隐藏层的净值为$z^1$, 激活值为$a^1$；输出层的净值为$z^L$, 激活值为$a^L$，即隐藏层加输出层的数量为L。第l层的神经元个数为$M_l$。

$w_{jk}^l$表示第$(l-1)$层的第$k$个神经元到$l$层的第$j$个神经元的连接上的权重，l-1和l层间的权重矩阵为$w^l\in \mathbb{R}^{\mathbf{M}_l\times \mathbf{M}_{l-1}}$。

$b_j^l$表示第l-1层到第l层第j个神经元的偏置, l层的偏置向量表示为$b^l\in \mathbb{R}^{\mathbf{M}_l}$。

$z_j^l$表示第l层第j个神经元的净值, l层的净值向量表示为$z^l\in \mathbb{R}^{\mathbf{M}_l}$。

$\sigma^l$表示应用于第l层神经元净值的激活函数。

$a_j^l$表示第l层第j个神经元的激活值, l层的激活值表示为$a^l\in \mathbb{R}^{\mathbf{M}_l}$。

$$
\begin{aligned}
z_j^l &= \sum_k w_{jk}^la_k^{l-1}+b_j^l \\
a_j^l &= \sigma^l(z_j^l)\\
\text{或者表示为向量矩阵形式}\\
z^l &= w^la^{l-1}+b^l \\
a^l &= \sigma^l(z^l)\\
\end{aligned}
$$

<div align=center>
<img width="1200" src="../pictures/7.computational_graph.svg"/>
</div>

<div align=center>
    图2 计算图
</div>

## 1. 反向传播的四个基本方程

$$
\begin{aligned}
\delta^L&=\frac{\partial{C}}{\partial{a^l}}\odot R'(z^L) \\
\delta^l&=((w^{l+1})^T\delta^{l+1})\odot\sigma'(z^l) \\
\frac{\partial{C}}{\partial{b_j^l}}&=\delta_j^l \text{ or } \frac{\partial{C}}{\partial{b^l}}=\delta^l \in \mathbb{R}^{M_l}\\
\frac{\partial{C}}{\partial{w_{jk}^l}}&=a_k^{l-1}\delta_j^l \text{ or } \frac{\partial{C}}{\partial{w^l}}=\delta^l (a^{l-1})^T\in \mathbb{R}^{M_l\times M_{l-1}}
\end{aligned}
$$

反向传播其实是对权重和偏置变化影响代价函数过程的理解。最终极的含义其实就是计算偏导数$\frac{\partial{C}}{\partial{w^l_{jk}}}$ 和$\frac{\partial{C}}{\partial{b^l}}$。但是为了计算这些值，我们⾸先引⼊⼀个中间量，$\delta_j^l$，这个我们称为第l层第j个神经元上的误差。

反向传播将给出计算误差$\delta_j^l$的流程，然后将其关联到计算$\frac{\partial{C}}{\partial{w^l_{jk}}}$ 和$\frac{\partial{C}}{\partial{b^l}}$上。

<div align=center>
<img width="800" src="../pictures/7.error_z.svg"/>
</div>

<div align=center>
    图3 误差图
</div>

假定在前馈信息传播过程中，l层第i个神经元的净值$z_i^l$增加了$\Delta z_i^l$，则最终损失函数的增量应为$\frac{\partial C}{\partial z_i^l}\Delta z_i^l$。

假设$\frac{\partial C}{\partial z_i^l}$有⼀个很⼤的值（或正或负）。那么可以通过选择与$\frac{\partial C}{\partial z_i^l}$相反符号的$\Delta z_i^l$来降低代价。相反，如果$\frac{\partial C}{\partial z_i^l}$接近0，那么我们并不能通过扰动带权输⼊$z_j^l$来改善太多代价。这时候神经元已经很接近最优了2。所以这⾥有⼀种启发式的认识， $\frac{\partial C}{\partial z_j^l}$是l层神经元j的误差的度量。

按照上⾯的描述，我们定义l层的第j个神经元上的误差$\delta_i^l$为：
$$
\delta_j^l = \frac{\partial C}{\partial z_j^l}
$$

按照我们通常的惯例，我们使⽤$\delta^l$表⽰关联于l 层的误差向量。反向传播会提供给我们⼀种计算每层的$\delta^l$的⽅法，然后将这些误差和最终我们需要的量$\frac{\partial C}{\partial w_{jk}^l}$和$\frac{\partial C}{\partial b_j^l}$联系起来。

- (BP1). 输出层的误差项$\delta^L$
每个元素定义如下：
$$
\delta_j^L=\frac{\partial C}{\partial a_j^L}\sigma'(z_j^L)
$$

这是⼀个⾮常⾃然的表达式。右式第⼀个项$\frac{\partial C}{\partial a_j^L}$表⽰代价随着j层输出激活值的变化⽽变化的速度。假如C不太依赖⼀个特定的输出神经元j，那么$\frac{\partial C}{\partial a_j^L}$就会很⼩，这也是我们想要的效果。右式第⼆项$\sigma'(z_j^L)$刻画了在$z_j^L$处激活函数$\sigma$变化的速度。

上式也可以重新写成矩阵形式
$$
\delta^L=\Delta_aC\odot R'(z^L)
$$

例如，如果损失函数为误差平方和时，我们有$\Delta_aC=(a^L-y)$，因此可得$\delta_L=(a^L-y)\odot \sigma'(z^L)$

- (BP2). 使用l+1层误差$\delta^{l+1}$计算l层的误差$\delta^{l}$: 
$$
\delta^l=((w^{l+1})^T\delta^{l+1})\odot\sigma'(z^l)
$$

假设我们知道l+1层的误差$\delta^{l+1}$。当我们应⽤转置的权重矩阵$(w^{l+1})^T$ ，我们可以凭直觉地把它看作是在沿着⽹络反向移动误差，给了我们度量在l层输出的误差⽅法。然后，我们进⾏Hadamard 乘积运算$\odot\sigma'(z^l)$。这会让误差通过l 层的激活函数反向传递回来并给出在第l 层的带权输⼊的误差$\delta$。

**通过组合(BP1) 和(BP2)，我们可以计算任何层的误差$\delta^l$。⾸先使⽤(BP1) 计算$\delta^L$，然后应⽤⽅程(BP2) 来计算$\delta^{L-1}$，然后再次⽤⽅程(BP2)来计算$\delta^{L-2}$，如此⼀步⼀步地反向传播完整个⽹络。**

- (BP3). 代价函数关于⽹络中任意偏置的改变率:
$$
\frac{\partial{C}}{\partial{b_j^l}}=\delta_j^l \\
\text{ or } \\
\frac{\partial{C}}{\partial{b^l}}=\delta^l \in \mathbb{R}^{M_l}
$$

<div align=center>
<img width="800" src="../pictures/7.bp3-4.svg"/>
</div>

<div align=center>
    图4 BP(3)-(4)示意图
</div>

- (BP4). 代价函数关于权重的改变率
$$
\frac{\partial{C}}{\partial{w_{jk}^l}}=a_k^{l-1}\delta_j^l \\
\text{ or } \\
\frac{\partial{C}}{\partial{w^l}}=\delta^l (a^{l-1})^T\in \mathbb{R}^{M_l\times M_{l-1}}
$$

当激活值$a_k^{l-1}$很⼩，例如$a_k^{l-1}\simeq 0$时，梯度$\frac{\partial{C}}{\partial{w_{jk}^l}}$也会趋向很⼩。这
样，我们就说权重缓慢学习，表⽰在梯度下降的时候，这个权重不会改变太多。换⾔之，(BP4)的⼀个结果就是来⾃低激活值神经元的权重学习会⾮常缓慢。

- 交叉熵风险函数$\mathbf{R(z^L, y)}=-y\cdot\log{a^L}$关于$z^L$的梯度为
> $a^L=\mathrm{softmax}(z^L), \frac{\partial{C}}{\partial{a_i^L}}=-y_i\frac{1}{a_i^L}$
>
> $\frac{\partial{C}}{\partial{a^L}} = -y\odot (a^L)^{-1} = (-y_1\frac{1}{a_1^L}, -y_2\frac{1}{a_2^L}, ..., -y_{M_L}\frac{1}{a_{M_L}^L})$

当$i=j$时, 有$\frac{\partial{a_i}}{\partial{b_j}}=a_i(1-a_i)$; 当$i\neq j$时, 有$\frac{\partial{a_i}}{\partial{b_j}}=-a_ia_j$, 因此有

$$
\frac{\partial{a^L}}{\partial{z^L}} = \begin{bmatrix} 
a_1^L(1-a_1^L) & -a_1^La_2^L & ... & -a_1^La_j^L & ... & -a_1^La_{M_L}^L \\ 
-a_2^La_1^L & a_2^L(1-a_2^L) & ... & a_2^La_j^L & ... & -a_2^La_{M_L}^L \\
... & ... & ... & ... & ... & ....\\
-a_{M_L}^La_1^L & -a_{M_L}^La_2^L & ... & -a_{M_L}^La_j^L & ... & a_{M_L}^L(1-a_{M_L}^L)
\end{bmatrix}
$$

因此，结合链式法则，有
$$
\begin{aligned}
\frac{\partial{C}}{\partial{z^L}} &= \frac{\partial{C}}{\partial{a^L}} \frac{\partial{a^L}}{\partial{z^L}} \\
&= (\frac{\partial{C}}{\partial{a^L}}\frac{\partial{a^L}}{\partial{z^L_1}}, \frac{\partial{C}}{\partial{a^L}}\frac{\partial{a^L}}{\partial{z^L_2}}, ..., \frac{\partial{C}}{\partial{a^L}}\frac{\partial{a^L}}{\partial{z^L_{M_L}}})
\end{aligned}
$$

假定$y_i$=1, 则有$\frac{\partial{C}}{\partial{z^L}}=(a^L_1, a^L_2, ..., a^L_{i}-1, ..., a^L_{M_L})$, 因此可以写成: $\frac{\partial{C}}{\partial{z^L}}=a^L-y$

## 2. 训练深度学习模型

反向传播方程给出了一种计算代价函数梯度的方法，可以用以下算法描述:

**输入: 训练集{features, labels}, 训练回合数max_epochs, 学习率lr, 批量大小batch_size**
**输出: 训练好的前馈神经网络**
**算法过程:**
- 初始化当前回合epoch=1, 
- 如果epoch <= max_epochs, 执行以下操作
    - 打乱训练集的排序
    - 由前逐批取出batch_size个样本，然后做以下计算，直到取完所有样本为止
        - 前馈计算每一层的净值$z^l$和激活值$a^l$，直到最后一层
        - 反向传播计算每一层的误差$\delta^l$(*公式bp1, bp2*)
        - 计算损失函数对各层间权重矩阵$w^l$和偏置向量$b^l$的偏导数(*公式bp3, bp4*)
        - 更新权重矩阵和权重矩阵
        $$
        \begin{aligned}
        w^l &:= w^l - lr*\frac{(\delta^l(a(l-1)^T))}{\mathrm{batch\_size}} \\
        b^l &:= b^l - lr*mean(\delta^l, axis=1)
        \end{aligned}
        $$
    - 更新epoch := epoch + 1

In [2]:
class FNN:
    def __init__(self, features, labels, params, batch_size=256):
        '''
        features: 特征
        labels: 标签
        params元素: (权重矩阵, 偏置向量, 激活函数, 激活函数的导数)
        注意: 与以上符号保持一致，权重的形状为 (M_{l}, M_{l-1}), 偏置的形状为 (1, M_{l})
        '''
        self.features = features
        self.labels = labels
        self.params = params
        self.train_iter = self.data_loader(batch_size=256)
    
    def data_loader(self, batch_size, is_one_hot=True):
        '''
        构建小批量数据集
        '''
        if is_one_hot:
            hot_labels = torch.zeros(self.features.shape[0], 10)
            x_indices = np.arange(self.features.shape[0]).tolist()
            y_indices = self.labels.byte().tolist()
            hot_labels[x_indices, y_indices] = 1
            dataset = TensorDataset(self.features, hot_labels)
        else:
            dataset = TensorDataset(self.features, self.labels)

        return DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)
    
    def forward(self, X):
        '''
        神经元前馈传递信息
        '''
        self.z_list, self.a_list = [], [X]  # 记录各层的净值和激活值
        y = X  # 初始化输入features
        for weight, bias, func, _ in self.params:
            z = y@torch.transpose(weight, 0, 1) + bias.reshape(1, -1)  # (N, M_{l-1}) @ (M_{l-1}, M_{l}) + (1, M_{l}), broadcast
            if func:
                if func.__name__ == 'softmax':
                    y = func(z, dim=1)
                else:
                    y = func(z)
            else:
                y = z
                
            self.z_list.append(z)
            self.a_list.append(y)
        return y.double()
    
    def cal_neuron_errors(self, y):
        '''
        计算神经元的误差delta
        '''
        # 输出层误差
        error_L = self.a_list[-1] - y
        self.error_list = [error_L]
        for i in range(len(self.params)-1):  # 从输出层至输入层，逐层计算
            weight = self.params[-i-1][0]  # 权重矩阵
            der_f = self.params[-i-1][-1]  # 激活函数的导数
            error_up = self.error_list[-1]  # 上一层的误差
            z = self.z_list[-i-2]  # 当前层的净值
            error = error_up@weight * der_f(z)  #  (N, M_{l})@(M_{l}, M_{l-1}) = (N, M_{l-1})
            self.error_list.append(error)
            
        self.error_list.reverse()  # 逆序，为了后面的更新
    
    def cal_params_partial(self):
        '''
        计算损失函数关于权重和偏置的偏导数
        '''
        self.der_weight_list = []  # 权重梯度
        self.der_bias_list = []  # 偏置梯度
        for i in range(len(self.params)):
            a_out = self.a_list[i]  # l-1层激活值
            error_in = self.error_list[i]  # l层误差
            # 以下计算出来的是每个样本对应的der_weight构成的矩阵，归约成1维，可采用均值或求和的形式
            der_weight = torch.transpose(error_in, 0, 1)@a_out / self.a_list[0].shape[0]  # (M_{l}, N) @ (N, M{l-1})
            der_bias = torch.mean(torch.transpose(error_in, 0, 1), axis=1)  # (M_{l}, N)
            self.der_weight_list.append(der_weight)
            self.der_bias_list.append(der_bias)
        
    def backward(self, y):
        '''
        误差反向传播算法实现
        '''
        self.cal_neuron_errors(y)
        self.cal_params_partial()
    
    def cross_entropy(self, X, y):
        '''
        采用交叉熵损失函数
        labels: one-hot形式
        hat_y: softmax之后对应概率向量，多层感知机的输出
        '''
        hat_y = self.forward(X)
        if len(y.shape) == 2:
            crossEnt = -torch.dot(y.reshape(-1), torch.log10(hat_y.float()).reshape(-1)) / y.shape[0]  # 展开成1维，点积
        elif len(y.shape) == 1:
            crossEnt = -torch.mean(torch.log10(hat_y[torch.arange(y.shape[0]), y.long()]))
        else:
            print("Wrong format of y!")
        return crossEnt
    
    def accuracy(self, y, hat_y, is_one_hot=False):
        '''
        y: 标签, one-hot
        hat_y: 标签预测概率, one-hot
        is_one_hot: y是否为one-hot形式
        '''
        if is_one_hot:
            precision = torch.sum(torch.max(y, axis=1)[1] == torch.max(hat_y, axis=1)[1]).numpy() / y.shape[0]
        else:
            precision = torch.sum((y == torch.max(hat_y, axis=1)[1]).byte()).numpy() / y.shape[0]
        return precision
    
    def minibatch_sgd_trainer(self, max_epochs=10, lr=0.1):
        '''
        训练
        '''
        for epoch in range(max_epochs):
            for X, y in self.train_iter:
                self.forward(X)  # 前向传播
                self.backward(y)  # 误差反向传播
                for i in range(len(self.params)):  # 更新权重
                    self.params[i][0] -= lr*self.der_weight_list[i]
                    self.params[i][1] -= lr*self.der_bias_list[i]
            
            loss = self.cross_entropy(self.features, self.labels)
            accu = self.accuracy(self.labels, self.forward(self.features))
            print(f"第{epoch+1}个回合, 训练集交叉熵损失为:{loss:.4f}, 分类准确率{accu:.4f}")

In [3]:
def d_relu(x):
    '''
    relu激活函数的导数
    '''
    d = torch.zeros_like(x)
    d[x > 0] = 1
    return d

In [4]:
def d_softmax(x):
    '''
    softmax激活函数的导数
    '''
    d = torch.softmax(x, dim=1)
    return d*(1-d)

## 3. 案例-`fashion-mnist`数据集

- 装载数据

In [5]:
def load_mnist(path, kind='train'):
    """
    Load MNIST data from `path`
    """
    labels_path = os.path.join(path, '%s-labels-idx1-ubyte.gz'% kind)
    images_path = os.path.join(path, '%s-images-idx3-ubyte.gz'% kind)

    with gzip.open(labels_path, 'rb') as lbpath:
        labels = np.frombuffer(lbpath.read(), dtype=np.uint8, offset=8)

    with gzip.open(images_path, 'rb') as imgpath:
        images = np.frombuffer(imgpath.read(), dtype=np.uint8, offset=16).reshape(len(labels), 784)
    
    features = transforms.ToTensor()(images)  # (h, w, c) -> (c, h, w)
    labels = torch.LongTensor(labels)

    return features[0], labels

In [6]:
label_names = ['短袖圆领T恤', '裤子', '套衫', '连衣裙', '外套', '凉鞋', '衬衫', '运动鞋','包', '短靴']
text_labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']

In [7]:
features, labels = load_mnist(path="../dataset/fashion_mnist")
test_features, test_labels = load_mnist(path="../dataset/fashion_mnist", kind="t10k")

/Users/xiaoyu/miniforge3/envs/pyg_env/lib/python3.10/site-packages/torchvision/transforms/functional.py:154: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/torch/csrc/utils/tensor_numpy.cpp:212.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


- 训练

In [8]:
num_inputs, num_outputs, num_hiddens = 784, 10, 256
W1 = torch.tensor(np.random.normal(0, 0.01, (num_hiddens, num_inputs)), dtype=torch.float)
b1 = torch.zeros(num_hiddens, dtype=torch.float)
W2 = torch.tensor(np.random.normal(0, 0.01, (num_outputs, num_hiddens)), dtype=torch.float)
b2 = torch.zeros(num_outputs, dtype=torch.float)
init_params = [[W1, b1, torch.relu, d_relu], [W2, b2, torch.softmax, d_softmax]]

fnn = FNN(features, labels, init_params)

In [9]:
fnn.minibatch_sgd_trainer(max_epochs=40, lr=0.99)

第1个回合, 训练集交叉熵损失为:0.2958, 分类准确率0.7429
第2个回合, 训练集交叉熵损失为:0.2721, 分类准确率0.7674
第3个回合, 训练集交叉熵损失为:0.2270, 分类准确率0.8141
第4个回合, 训练集交叉熵损失为:0.2080, 分类准确率0.8327
第5个回合, 训练集交叉熵损失为:0.2264, 分类准确率0.8076
第6个回合, 训练集交叉熵损失为:0.2261, 分类准确率0.8079
第7个回合, 训练集交叉熵损失为:0.1966, 分类准确率0.8410
第8个回合, 训练集交叉熵损失为:0.1944, 分类准确率0.8413
第9个回合, 训练集交叉熵损失为:0.1985, 分类准确率0.8393
第10个回合, 训练集交叉熵损失为:0.1855, 分类准确率0.8501
第11个回合, 训练集交叉熵损失为:0.1737, 分类准确率0.8613
第12个回合, 训练集交叉熵损失为:0.1983, 分类准确率0.8339
第13个回合, 训练集交叉熵损失为:0.1757, 分类准确率0.8584
第14个回合, 训练集交叉熵损失为:0.1696, 分类准确率0.8606
第15个回合, 训练集交叉熵损失为:0.1710, 分类准确率0.8617
第16个回合, 训练集交叉熵损失为:0.1733, 分类准确率0.8581
第17个回合, 训练集交叉熵损失为:0.1738, 分类准确率0.8591
第18个回合, 训练集交叉熵损失为:0.1743, 分类准确率0.8552
第19个回合, 训练集交叉熵损失为:0.1678, 分类准确率0.8606
第20个回合, 训练集交叉熵损失为:0.1628, 分类准确率0.8673
第21个回合, 训练集交叉熵损失为:0.1595, 分类准确率0.8697
第22个回合, 训练集交叉熵损失为:0.1619, 分类准确率0.8662
第23个回合, 训练集交叉熵损失为:0.1640, 分类准确率0.8643
第24个回合, 训练集交叉熵损失为:0.1729, 分类准确率0.8518
第25个回合, 训练集交叉熵损失为:0.1657, 分类准确率0.8649
第26个回合, 训练集交叉熵损失为:0.1577, 分类准确率0.8710
第27个回合, 训练集交叉熵损失为:0.1

In [10]:
test_crossEn = fnn.cross_entropy(test_features, test_labels)
test_accu = fnn.accuracy(test_labels, fnn.forward(test_features), is_one_hot=False)
print(f"测试集上的交叉熵为{test_crossEn:.4f}, 测试集的准确率为:{test_accu:.4f}")

测试集上的交叉熵为0.1884, 测试集的准确率为:0.8445


In [11]:
p = 0.9
succ_list = []
n = 10000
num = 0
for i in range(n):
    if p > np.random.rand():
        succ_list.append(i)
        num += 1
        
print(num / n)

0.9013


## 4. 初始化和正则化

### 4.1 参数初始化

神经网络的参数学习是一个非凸优化问题．当使用梯度下降法来进行优化网络参数时，参数初始值的选取十分关键，关系到网络的优化效率和泛化能力．

参数初始化的方式通常有以下三种：

- 预训练初始化：不同的参数初始值会收敛到不同的局部最优解．虽然这些局部最优解在训练集上的损失比较接近，但是它们的泛化能力差异很大．一个好的初始值会使得网络收敛到一个泛化能力高的局部最优解．通常情况下，一个已经在大规模数据上训练过的模型可以提供一个好的参数初始值，这种初始化方法称为**预训练初始化(`Pre-trained Initialization`)**．预训练任务可以为监督学习或无监督学习任务．由于无监督学习任务更容易获取大规模的训练数据，因此被广泛采用．预训练模型在目标任务上的学习过程也称为**精调(`Fine-Tuning`)**．
- 随机初始化：在线性模型的训练（比如感知器和Logistic 回归）中，我们一般将参数全部初始化为0．但是这在神经网络的训练中会存在一些问题．因为如果参数都为0，在第一遍前向计算时，所有的隐藏层神经元的激活值都相同；在反向传播时，所有权重的更新也都相同，这样会导致隐藏层神经元没有区分性．这种现象也称为**对称权重现象**．为了打破这个平衡，比较好的方式是对每个参数都**随机初始化(`Random Initialization`)**，使得不同神经元之间的区分性更好．
- 固定值初始化：对于一些特殊的参数，我们可以根据经验用一个特殊的固定值来进行初始化．比如偏置(Bias)通常用0来初始化，但是有时可以设置某些经验值以提高优化效率．对于使用ReLU 的神经元，有时也可以将偏置设为0.01，使得ReLU 神经元在训练初期更容易激活，从而获得一定的梯度来进行误差反向传播．

虽然预训练初始化通常具有更好的收敛性和泛化性，但是灵活性不够，不能在目标任务上任意地调整网络结构．因此，好的随机初始化方法对训练神经网络模型来说依然十分重要．这里我们介绍三类常用的随机初始化方法：
- 基于固定方差的参数初始化
- 基于方差缩放的参数初始化
- 正交初始化方法．

In [12]:
import torch.nn.init as init
import torch.nn as nn

In [13]:
linear = nn.Linear(4, 5)

In [14]:
def zero_init(m):
    if isinstance(m, nn.Linear):
        init.zeros_(m.weight)
        if m.bias is not None:
            init.zeros_(m.bias)

def random_small_init(m, scale=0.01):
    if isinstance(m, nn.Linear):
        init.normal_(m.weight, mean=0., std=scale)
        if m.bias is not None:
            init.normal_(m.bias, mean=0., std=scale)

In [15]:
zero_init(linear)
linear.weight

Parameter containing:
tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]], requires_grad=True)

In [16]:
random_small_init(linear)
linear.weight

Parameter containing:
tensor([[ 0.0051,  0.0157,  0.0030, -0.0016],
        [ 0.0133,  0.0264, -0.0001,  0.0176],
        [-0.0156, -0.0055,  0.0014,  0.0201],
        [ 0.0014, -0.0157, -0.0102, -0.0113],
        [-0.0110,  0.0058,  0.0138,  0.0004]], requires_grad=True)

#### 4.4.1 基于固定方差的参数初始化

- 高斯分布初始化：使用一个高斯分布$N(0,\sigma^2)$对每个参数进行随机初始化．
- 均匀分布初始化：在一个给定的区间$[-r, r]$内采用均匀分布来初始化参数．假设随机变量x在区间$[a, b]$内均匀分布，则其方差为$var(x)=\frac{(b-a)^2}{12}$. 因此，若使用区间为$[-r, r]$的均分分布来采样，并满足$var(x)=\sigma^2$时，则𝑟的取值为$r=\sqrt{3\sigma^2}$

In [17]:
def uniform_init(m, limit=0.1):
    if isinstance(m, nn.Linear):
        init.uniform_(m.weight, -limit, limit)
        if m.bias is not None:
            init.uniform_(m.bias, -limit, limit)

def normal_init(m, mean=0.0, std=0.05):
    if isinstance(m, nn.Linear):
        init.normal_(m.weight, mean, std)
        if m.bias is not None:
            init.normal_(m.bias, mean, std)

In [18]:
uniform_init(linear)
linear.weight

Parameter containing:
tensor([[ 0.0966, -0.0815,  0.0951, -0.0038],
        [ 0.0082, -0.0410,  0.0604, -0.0348],
        [ 0.0560, -0.0570,  0.0083, -0.0569],
        [ 0.0684,  0.0224, -0.0690, -0.0007],
        [-0.0090,  0.0278, -0.0242,  0.0561]], requires_grad=True)

In [19]:
normal_init(linear)
linear.weight

Parameter containing:
tensor([[ 0.0252,  0.0729,  0.0522,  0.0355],
        [-0.0187,  0.0922, -0.0352,  0.0004],
        [-0.0791, -0.0533,  0.0084, -0.0010],
        [-0.0042,  0.0024,  0.0125,  0.0231],
        [ 0.0244, -0.0016,  0.0383, -0.0605]], requires_grad=True)

#### 4.4.2 基于方差缩放的参数初始化

| 初始化方法 | 激活函数 | 均匀分布$[-r, r]$ | 高斯分布 $N(0, \sigma^2)$ |
| ----: | ----: | ----: | ----: |
| Xavier | Logistic | $r=4\sqrt{\frac{6}{M_{l-1}+M_l}}$ | $\sigma^2=16\times \frac{2}{M_{l-1}+M_l}$ |
| Xavier | tanh | $r=\sqrt{\frac{6}{M_{l-1}+M_l}}$ | $\sigma^2=\frac{2}{M_{l-1}+M_l}$ |
| He | reLu | $r=\sqrt{\frac{6}{M_{l-1}}}$ | $\sigma^2=\frac{2}{M_{l-1}}$ |

In [20]:
def xavier_uniform_init(m):
    if isinstance(m, nn.Linear):
        init.xavier_uniform_(m.weight)
        if m.bias is not None:
            init.zeros_(m.bias)

def xavier_normal_init(m):
    if isinstance(m, nn.Linear):
        init.xavier_normal_(m.weight)
        if m.bias is not None:
            init.zeros_(m.bias)

In [21]:
xavier_uniform_init(linear)
linear.weight

Parameter containing:
tensor([[ 0.2737, -0.6806, -0.5349, -0.7650],
        [ 0.7183,  0.4407, -0.4204,  0.2533],
        [ 0.4537,  0.7934, -0.4715, -0.7976],
        [-0.4746,  0.3190, -0.3391, -0.1280],
        [ 0.6417, -0.5027,  0.7957,  0.4687]], requires_grad=True)

In [22]:
def he_uniform_init(m):
    if isinstance(m, nn.Linear):
        init.kaiming_uniform_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            init.zeros_(m.bias)

def he_normal_init(m):
    if isinstance(m, nn.Linear):
        init.kaiming_normal_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            init.zeros_(m.bias)

In [23]:
he_normal_init(linear)
linear.weight

Parameter containing:
tensor([[-0.7659,  0.4525, -0.8623,  0.0508],
        [-0.4903, -0.4842, -0.5590, -0.0270],
        [-1.8107,  0.5373, -1.7465,  0.3396],
        [ 1.5636,  0.0913,  0.7725, -1.4994],
        [-0.0165, -0.7294, -0.6631, -0.3167]], requires_grad=True)

### 4.4.3 正交初始化

用于保持每层输入的特征分布在前向传播和反向传播时保持不变。具体来说，对于每一层的权重矩阵进行正交化处理。这种方法特别适合于循环神经网络（RNN），可以有效防止梯度消失问题。

In [24]:
def orthogonal_init(m):
    if isinstance(m, nn.Linear):
        init.orthogonal_(m.weight)
        if m.bias is not None:
            init.zeros_(m.bias)

In [25]:
orthogonal_init(linear)
linear.weight

Parameter containing:
tensor([[-0.3259,  0.0148,  0.8493,  0.4147],
        [-0.8332, -0.0798, -0.3650,  0.1123],
        [-0.1414,  0.7852, -0.2349,  0.3226],
        [ 0.0054,  0.6042,  0.2626, -0.5326],
        [-0.4238, -0.1090,  0.1462, -0.6540]], requires_grad=True)

### 4.2 正则化

#### 4.2.1 $l_1$和$l_2$正则化
**ℓ1和ℓ2** 正则化是机器学习中最常用的正则化方法，通过约束参数的ℓ1 和ℓ2范数来减小模型在训练数据集上的过拟合现象.
$$
\theta^*=\text{arg}\min \frac{1}{N}\sum_{n=1}^N \mathbf{L}(y^{(n)},f(x^{(n)};\theta))+\lambda \mathrm{ℓ_p}(\theta)
$$
其中$\mathbf{L(\cdot)}$为损失函数，N为训练样本，$f(\cdot)$为待学习的神经网络，$\theta$为参数，$ℓ_p$为范数函数，𝑝 的取值通常为{1, 2} 代表$ℓ_1$ 和$ℓ_2$ 范数，𝜆 为正则化系数．

#### 4.2.2 权重衰减

**权重衰减(Weight Decay)** 是一种有效的正则化方法(`Hanson et al., 1989`)，在每次参数更新时，引入一个衰减系数．
$$
\begin{aligned}
W^l &:= (1-\beta)W^l - \alpha \frac{\partial C}{\partial W^l}\\
b^l &:= (1-\beta)b^l - \alpha \frac{\partial C}{\partial b^l}
\end{aligned}
$$

其中$\alpha$为学习率，$\beta$为权重衰减系数，一般取值比较小，比如0.0005．在标准的随机梯度下降中，权重衰减正则化和$ℓ_2$正则化的效果相同．因此，权重衰减在一些深度学习框架中通过$ℓ_2$正则化来实现．但是，较为复杂的优化方法（比如Adam）中，权重衰减正则化和$ℓ_2$正则化并不等价.

#### 4.2.3 提前终止
**提前停止(Early Stop)** 对于深度神经网络来说是一种简单有效的正则化方法．由于深度神经网络的拟合能力非常强，因此比较容易在训练集上过拟合．在使用梯度下降法进行优化时，我们可以使用一个和训练集独立的样本集合，称为验证集（Validation Set），并用验证集上的错误来代替期望错误．当验证集上的错误率不再下降，就停止迭代．然而在实际操作中，验证集上的错误率变化曲线很可能是先升高再降低．因此，提前停止的具体停止标准需要根据实际任务进行优化(`Prechelt, 1998`)．

#### 4.2.4 丢弃法`dropout`

当训练一个深度神经网络时， 我们可以随机丢弃一部分神经元（同时丢弃其对应的连接边）来避免过拟合，这种方法称为**丢弃法（Dropout Method）** [Srivastava et al., 2014]．每次选择丢弃的神经元是随机的．最简单的方法是设置一个固定的概率𝑝．对每一个神经元都以概率𝑝 来判定要不要保留, 若$p=1$，则意味着需要保留该神经元；若$p=0$, 则意味着不保留该神经元．对于一个神经层$y=f(W@x + b)$，我们可以引入一个掩蔽函数`mask(⋅)` 使得$y=f(W@mask(x) + b)$．掩蔽函数mask(⋅) 的定义为
$$
\begin{equation}
\mathrm{mask}(x)=\begin{cases} m \odot x \text{  训练时}\\
px \text{  测试时}\\
\end{cases}
\end{equation}
$$
或者
$$
\begin{equation}
\mathrm{mask}(x)=\begin{cases} m \odot \frac{x}{p} \text{  训练时}\\
x \text{  测试时}\\
\end{cases}
\end{equation}
$$

In [26]:
# Dropout概率设置为0.5
dropout = nn.Dropout(p=0.1)
# 假设有一层的输出
layer_output = torch.randn(4, 5)  # 随机生成一些数据
# 应用Dropout

In [27]:
output_during_training = dropout(layer_output)
# 输出查看
print("Output with Dropout during training:")
print(output_during_training)

Output with Dropout during training:
tensor([[-2.3158,  1.2802,  0.6884,  1.1302,  0.8604],
        [ 0.8891,  0.8564,  2.3151,  0.5656,  0.2610],
        [-0.8781, -0.2319,  1.2873,  1.0985,  0.1818],
        [ 0.5214,  0.1659, -0.0000, -0.0000,  1.0159]])


In [28]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.layer1 = nn.Linear(784, 256)
        self.dropout1 = nn.Dropout(0.5)
        self.layer2 = nn.Linear(256, 10)
    
    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.dropout1(x)  # 在第一层后应用Dropout
        x = self.layer2(x)
        return x

# 注意：模型训练时会应用Dropout，测试时需关闭Dropout
model = MyModel()
model.train()  # 确保是训练模式

MyModel(
  (layer1): Linear(in_features=784, out_features=256, bias=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (layer2): Linear(in_features=256, out_features=10, bias=True)
)

> `model.train()`和`model.eval()`的区别在于训练时，模型中的dropout层会保留，而在测试时，dropout层会被移除．此外，对于一些特殊的层，比如Batch Normalization层，训练时和测试时的行为也是不同的．

In [29]:
a = torch.randint(1, high=10, size=(20, 5))
p = 0.3
b = p*torch.ones_like(a)
torch.bernoulli(b) / p

tensor([[0.0000, 3.3333, 0.0000, 3.3333, 3.3333],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [3.3333, 0.0000, 3.3333, 0.0000, 0.0000],
        [0.0000, 0.0000, 3.3333, 0.0000, 0.0000],
        [3.3333, 3.3333, 3.3333, 0.0000, 3.3333],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 3.3333],
        [3.3333, 3.3333, 0.0000, 0.0000, 0.0000],
        [3.3333, 3.3333, 3.3333, 3.3333, 0.0000],
        [0.0000, 0.0000, 3.3333, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 3.3333, 3.3333],
        [0.0000, 0.0000, 0.0000, 3.3333, 3.3333],
        [0.0000, 3.3333, 0.0000, 0.0000, 0.0000],
        [0.0000, 3.3333, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 3.3333, 0.0000, 0.0000],
        [3.3333, 0.0000, 0.0000, 3.3333, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 3.3333],
        [0.0000, 0.0000, 3.3333, 0.0000, 0.0000],
        [0.0000, 3.3333, 3.3333, 3.3333, 3.3333]])

In [30]:
class FNN2:
    def __init__(self, features, labels, params, prob_dropout, batch_size=256):
        '''
        features: 特征
        labels: 标签
        params元素: (权重矩阵, 偏置向量, 激活函数, 激活函数的导数)
        注意: 权重的形式为 (M_{l}, M_{l-1}), 偏置的形式为 (1, M_{l})
        '''
        self.features = features
        self.labels = labels
        self.params = params
        self.prob_dropout = prob_dropout  # 对隐藏层进行dropout操作
        self.train_iter = self.data_loader(batch_size=256)
    
    def data_loader(self, batch_size, is_one_hot=True):
        '''
        构建小批量数据集
        '''
        if is_one_hot:
            hot_labels = torch.zeros(self.features.shape[0], 10)
            x_indices = np.arange(self.features.shape[0]).tolist()
            y_indices = self.labels.byte().tolist()
            hot_labels[x_indices, y_indices] = 1
            dataset = TensorDataset(self.features, hot_labels)
        else:
            dataset = TensorDataset(self.features, self.labels)

        return DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)
    
    def mask(self, X, p):
        '''
        X: 输入
        p: 神经元的保留概率
        若输入X除以概率p，则使X的期望保持不变, 预测时,神经网络的权重不用转换；
        若训练时按正常的输入训练X，预测时神经网络中对应的权重需乘p
        '''
        if p == 0:
            return torch.zeros_like(X)
        elif p == 1:
            return X
        else:
            prob = p*torch.ones_like(X)
            return X*torch.bernoulli(prob)/p
    
    def train_forward(self, X):
        '''
        训练用神经元前馈传递信息: 依照概率p随机关闭一些神经元
        '''
        y = X  # 初始化输入features
        self.z_list, self.a_list = [], [y]  # 记录各层的净值和激活值
        for i, (weight, bias, func, _) in enumerate(self.params, start=1):
            p = self.prob_dropout[i]
            z = y@torch.transpose(weight, 0, 1) + bias.reshape(1, -1)  # (N, M_{l-1}) @ (M_{l-1}, M_{l}) + (1, M_{l}), broadcast
            mask_z = self.mask(z, p)
            if func:
                if func.__name__ == 'softmax':
                    y = func(mask_z, dim=1)
                else:
                    y = func(mask_z)
            else:
                y = mask_z
            
            self.z_list.append(mask_z)
            self.a_list.append(y)
        return y.double()
    
    def predict_forward(self, X):
        '''
        预测用神经元前馈传递信息: 使用所有神经元
        '''
        y = X  # 初始化输入features
        for i, (weight, bias, func, _) in enumerate(self.params):
            z = y@torch.transpose(weight, 0, 1) + bias.reshape(1, -1)
            if func:
                if func.__name__ == 'softmax':
                    y = func(z, dim=1)
                else:
                    y = func(z)
            else:
                y = z

        return y.double()
    
    def cal_neuron_errors(self, y):
        '''
        计算神经元的误差
        '''
        # 输出层误差
        error_L = self.a_list[-1] - y
        self.error_list = [error_L]
        for i in range(len(self.params)-1):
            weight = self.params[-i-1][0]  # 权重矩阵
            der_f = self.params[-i-1][-1]  # 导数
            error_up = self.error_list[-1]  # 上一层的误差
            z = self.z_list[-i-2]  # 当前层的净值
            error = error_up@weight * der_f(z)  #  (N, M_{l})@(M_{l}, M_{l-1}) = (N, M_{l-1})
            self.error_list.append(error)
            
        self.error_list.reverse()
    
    def cal_params_partial(self):
        '''
        计算损失函数关于权重和偏置的偏导数
        '''
        self.der_weight_list = []
        self.der_bias_list = []
        for i in range(len(self.params)):
            a_out = self.a_list[i]
            error_in = self.error_list[i]
            # 以下计算出来的是每个样本对应的der_weight构成的矩阵，归约成1维，可采用均值或求和的形式
            der_weight = torch.transpose(error_in, 0, 1)@a_out / self.a_list[0].shape[0]  # (M_{l}, N) @ (N, M{l-1})
            der_bias = torch.mean(torch.transpose(error_in, 0, 1), axis=1)  # (M_{l}, N)
            self.der_weight_list.append(der_weight)
            self.der_bias_list.append(der_bias)
        
    def backward(self, y):
        '''
        误差反向传播算法实现
        '''
        self.cal_neuron_errors(y)
        self.cal_params_partial()

    def cross_entropy(self, y, hat_y):
        '''
        采用交叉熵损失函数
        y: one-hot形式
        hat_y: softmax之后对应概率向量，多层感知机的输出
        '''
        if len(y.shape) == 2:
            crossEnt = -torch.dot(y.reshape(-1), torch.log10(hat_y.float()).reshape(-1)) / y.shape[0]  # 展开成1维，点积
        elif len(y.shape) == 1:
            crossEnt = -torch.mean(torch.log10(hat_y[torch.arange(y.shape[0]), y.long()]))
        else:
            print("Wrong format of y!")
        return crossEnt
    
    def accuracy(self, y, hat_y, is_one_hot=False):
        '''
        y: 标签, one-hot
        hat_y: 标签预测概率, one-hot
        is_one_hot: y是否为one-hot形式
        '''
        if is_one_hot:
            precision = torch.sum(torch.max(y, axis=1)[1] == torch.max(hat_y, axis=1)[1]).numpy() / y.shape[0]
        else:
            precision = torch.sum((y == torch.max(hat_y, axis=1)[1]).byte()).numpy() / y.shape[0]
        return precision
    
    def minibatch_sgd_trainer(self, max_epochs=10, lr=0.1, decay=0.0005):
        '''
        训练
        lr: 学习率
        decay: 权重衰减系数
        '''
        for epoch in range(max_epochs):
            for X, y in self.train_iter:
                self.train_forward(X)  # 前向传播
                self.backward(y)  # 误差反向传播
                for i in range(len(self.params)):
                    self.params[i][0] = (1 - decay)*self.params[i][0] - lr*self.der_weight_list[i]
                    self.params[i][1] = (1 - decay)*self.params[i][1] - lr*self.der_bias_list[i]
            
            hat_labels = self.predict_forward(self.features)
            loss = self.cross_entropy(self.labels, hat_labels)
            accu = self.accuracy(self.labels, hat_labels)
            print(f"第{epoch+1}个回合, 训练集交叉熵损失为:{loss:.4f}, 分类准确率{accu:.4f}")

In [31]:
num_inputs, num_outputs, num_hiddens = 784, 10, 256
W1 = torch.tensor(np.random.normal(0, 2/num_inputs, (num_hiddens, num_inputs)), dtype=torch.float)
b1 = torch.zeros(num_hiddens, dtype=torch.float)
W2 = torch.tensor(np.random.normal(0, 2/num_hiddens, (num_outputs, num_hiddens)), dtype=torch.float)
b2 = torch.zeros(num_outputs, dtype=torch.float)
init_params = [[W1, b1, torch.relu, d_relu], [W2, b2, torch.softmax, d_softmax]]
prob_dropout = [0.95, 0.5, 1]  # [输入层, 隐藏层, 输出层]

fnn2 = FNN2(features, labels, init_params, prob_dropout)

In [32]:
fnn2.minibatch_sgd_trainer(max_epochs=40, lr=0.1, decay=0.00001)

第1个回合, 训练集交叉熵损失为:0.9618, 分类准确率0.4646
第2个回合, 训练集交叉熵损失为:0.7850, 分类准确率0.5147
第3个回合, 训练集交叉熵损失为:0.5936, 分类准确率0.5990
第4个回合, 训练集交叉熵损失为:0.4973, 分类准确率0.6266
第5个回合, 训练集交叉熵损失为:0.4396, 分类准确率0.6800
第6个回合, 训练集交叉熵损失为:0.4021, 分类准确率0.6919
第7个回合, 训练集交叉熵损失为:0.3756, 分类准确率0.7115
第8个回合, 训练集交叉熵损失为:0.3583, 分类准确率0.7064
第9个回合, 训练集交叉熵损失为:0.3435, 分类准确率0.7259
第10个回合, 训练集交叉熵损失为:0.3325, 分类准确率0.7330
第11个回合, 训练集交叉熵损失为:0.3239, 分类准确率0.7321
第12个回合, 训练集交叉熵损失为:0.3198, 分类准确率0.7215
第13个回合, 训练集交叉熵损失为:0.3109, 分类准确率0.7416
第14个回合, 训练集交叉熵损失为:0.3086, 分类准确率0.7413
第15个回合, 训练集交叉熵损失为:0.3016, 分类准确率0.7461
第16个回合, 训练集交叉熵损失为:0.3013, 分类准确率0.7462
第17个回合, 训练集交叉熵损失为:0.2982, 分类准确率0.7511
第18个回合, 训练集交叉熵损失为:0.2916, 分类准确率0.7523
第19个回合, 训练集交叉熵损失为:0.2878, 分类准确率0.7600
第20个回合, 训练集交叉熵损失为:0.2865, 分类准确率0.7596
第21个回合, 训练集交叉熵损失为:0.2817, 分类准确率0.7589
第22个回合, 训练集交叉熵损失为:0.2794, 分类准确率0.7639
第23个回合, 训练集交叉熵损失为:0.2774, 分类准确率0.7664
第24个回合, 训练集交叉熵损失为:0.2772, 分类准确率0.7656
第25个回合, 训练集交叉熵损失为:0.2746, 分类准确率0.7610
第26个回合, 训练集交叉熵损失为:0.2721, 分类准确率0.7694
第27个回合, 训练集交叉熵损失为:0.2

In [33]:
hat_test_labels = fnn2.predict_forward(test_features)
test_crossEn = fnn2.cross_entropy(test_labels, hat_test_labels)
test_accu = fnn2.accuracy(test_labels, hat_test_labels, is_one_hot=False)
print(f"测试集上的交叉熵为{test_crossEn:.4f}, 测试集的准确率为:{test_accu:.4f}")

测试集上的交叉熵为0.2557, 测试集的准确率为:0.7890


- 由torch模块实现

In [34]:
from sklearn.metrics import accuracy_score

In [35]:
class Net(nn.Module):
    def __init__(self, dim_in=784, dim_hidden=256, dim_out=10, p=0.1):
        super(Net, self).__init__()
        self.layer1 = nn.Linear(dim_in, dim_hidden)
        self.dropout1 = nn.Dropout(p=p)
        self.layer2 = nn.Linear(dim_hidden, dim_out)
    
    def forward(self, x):
        x = torch.relu(self.layer1(x))
        x = self.dropout1(x)  # 在第一层后应用Dropout
        x = self.layer2(x)
        return x

In [36]:
dataloader = DataLoader(TensorDataset(features, labels), batch_size=256, shuffle=True)

In [37]:
# 构建训练实例
net = Net(p=0.1)
trainer = torch.optim.SGD(net.parameters(), lr=0.1, weight_decay=0.00001)
loss = nn.CrossEntropyLoss()

# 训练模式
net.train()
for i in range(20):
    for X, y in dataloader:
        trainer.zero_grad()
        l = loss(net(X), y)
        l.backward()
        trainer.step()
    
    if (i+1) % 5 == 0:
        net.eval()  # 评估模式
        with torch.no_grad():
            hat_labels = net(features)
            l = loss(hat_labels, labels)
            accu = accuracy_score(labels, torch.max(hat_labels, axis=1)[1])
            print(f"训练集交叉熵损失为:{l:.4f}, 分类准确率{accu:.4f}")
        
        net.train()

训练集交叉熵损失为:0.4142, 分类准确率0.8564
训练集交叉熵损失为:0.3611, 分类准确率0.8721
训练集交叉熵损失为:0.3551, 分类准确率0.8734
训练集交叉熵损失为:0.3178, 分类准确率0.8837


In [38]:
net.eval()  # 评估模式
with torch.no_grad():
    hat_test_labels = net(test_features)
    l = loss(hat_test_labels, test_labels)
    accu = accuracy_score(test_labels, torch.max(hat_test_labels, axis=1)[1])
    print(f"测试集交叉熵损失为:{l:.4f}, 分类准确率{accu:.4f}")

测试集交叉熵损失为:0.3786, 分类准确率0.8587


## 参考资料
1. 邱锡鹏. 神经网络与机器学习. 2020.
2. [阿斯顿·张、李沐、扎卡里 C. 立顿、亚历山大 J. 斯莫拉等. 动手学深度学习. 2020.](https://github.com/d2l-ai/d2l-zh)
3. [动手学深度学习(Pytoch实现)](https://github.com/ShusenTang/Dive-into-DL-PyTorch)
4. Michael Nielsen. Neural network and deep learning. 2016.